In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

In [10]:
newsgroups = datasets.fetch_20newsgroups(subset = 'all',
                    categories=['alt.atheism', 'sci.space'])

In [4]:
X = newsgroups.data
Y = newsgroups.target

In [8]:
tf_idf = TfidfVectorizer()
X_tr = tf_idf.fit_transform(X)

In [15]:
print X_tr.shape

(1786, 28382)


In [13]:
type(X_tr)

scipy.sparse.csr.csr_matrix

In [19]:
grid = {'C': np.power(10.0, np.arange(-5,6))}
cv = KFold(Y.size, n_folds=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring = 'accuracy', cv = cv)
gs.fit(X_tr, Y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [21]:
for item in gs.grid_scores_:
    print item.mean_validation_score # cross-validation quality estimation
    print item.parameters # parameters value
    
print gs.best_params_

0.552631578947
{'C': 1.0000000000000001e-05}
0.552631578947
{'C': 0.0001}
0.552631578947
{'C': 0.001}
0.552631578947
{'C': 0.01}
0.950167973124
{'C': 0.10000000000000001}
0.993281075028
{'C': 1.0}
0.993281075028
{'C': 10.0}
0.993281075028
{'C': 100.0}
0.993281075028
{'C': 1000.0}
0.993281075028
{'C': 10000.0}
0.993281075028
{'C': 100000.0}
{'C': 1.0}


In [22]:
print gs.best_estimator_

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)


In [28]:
raw = np.argsort(abs(gs.best_estimator_.coef_.data))
words = raw[-10:]
words

array([17469, 17422, 16746, 16935, 17536,  9015, 17045, 17449, 18092, 16693])

In [29]:
features = tf_idf.get_feature_names()
voc = []
for word in words:
    voc.append(features[gs.best_estimator_.coef_.indices[word]])
voc.sort()
print voc

[u'atheism', u'atheists', u'bible', u'god', u'keith', u'moon', u'religion', u'sci', u'sky', u'space']


In [32]:
file = open('words.txt', 'wt')
for i in range(len(voc)-1):
    file.write(voc[i] + ' ')
file.write(voc[-1])
file.close()